In [3]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import os

from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


## Carga de Dataset, Normalizacion y Aumento de Datos

In [5]:
base_directory = "/content/drive/MyDrive/proyectoai/cacao_diseases/cacao_photos"
image_size = 256
rotation_range = 40
width_move = 0.25
height_move = 0.25
tilt_range = 15
zoom_range = [0.5, 1]

In [4]:
train_datagenerator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = rotation_range,
    width_shift_range = width_move,
    height_shift_range = height_move,
    zoom_range = zoom_range,
    shear_range = tilt_range,
    validation_split = 0.2
)

test_datagenerator = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

In [6]:
train_data = train_datagenerator.flow_from_directory(
    base_directory,
    target_size = (image_size, image_size),
    batch_size = 32,
    subset = "training"
)

test_data = train_datagenerator.flow_from_directory(
    base_directory,
    target_size = (image_size, image_size),
    batch_size = 32,
    subset = "validation"
)

Found 3514 images belonging to 3 classes.
Found 876 images belonging to 3 classes.


## Creacion de Modelo

In [7]:
model = models.Sequential([
    layers.Conv2D(16, (3,3), input_shape=(image_size, image_size, 3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(units = 128, activation="relu"),
    layers.Dense(units = 50, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(units = 50, activation="relu"),
    layers.Dense(units = 3, activation="softmax")
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 conv2d_2 (Conv2D)           (None, 123, 123, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 61, 61, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 59, 59, 64)        1

In [9]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

## Entrenamiento

In [ ]:
epochs = 50
batch_size = 32

history = model.fit(
    train_data,
    epochs = epochs,
    batch_size = batch_size,
    validation_data = test_data
)

Epoch 1/50
 96/110 [=========================>....] - ETA: 2:07 - loss: 0.7190 - accuracy: 0.7305

## Evaluacion

In [ ]:
fig = plt.figure()
plt.plot(history.history["loss"], color = "teal", label = "loss")
plt.plot(history.history["val_loss"], color = "teal", label = "loss")
plt.subtitle('Loss', fontsize = 20)
plt.legend(loc = "upper left")
plt.show()

## Guardar Modelo

In [ ]:
model.save('proyectoai_g3.h5')